# 라이브러리 선언

In [85]:
# 스프레드시트의 행과열이 있는 데이터 조작 라이브러리
import pandas as pd
import numpy as np

# 머신러닝 모델 라이브러리
from sklearn import tree, ensemble, svm

# 시각화 라이브러리
import matplotlib.pyplot as plt

# 머신러닝 전처리 라이브러리
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 불러오기

In [ ]:
featureGitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/feature_regression_example.csv"
featuresData = pd.read_csv(featureGitUrl)
featuresData.head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442


# 1. 타입 통합/ 특성숫자컬럼 추가

In [10]:
### 테이블정의서 동일하게 타입을 정의해야 한다
### 특성 숫자 컬럼 추가

In [18]:
### np.where, function, loc
### 추가로 Label Encoder, Dictionary map 함수 써서 사용하는 방법!

In [40]:
# featuresData["LE_PROMOTION"] = np.where( featuresData.PROMOTION=="Y",1,0)
# featuresData["LE_HOLIDAY"] = np.where( featuresData.HOLIDAY=="Y",1,0)

In [41]:
ynDict = {"Y":1,"N":0}
featuresData["LE_PROMOTION"] = featuresData.PROMOTION.map( ynDict )
featuresData["LE_HOLIDAY"] = featuresData.HOLIDAY.map( ynDict )

In [24]:
### 라벨 인코더
# ynLabel = LabelEncoder()
# featuresData["LE_PROMOTION"] = ynLabel.fit_transform(featuresData.PROMOTION)
# # 각 데이터가 라벨링된 내용 확인 (카테고리 데이터를 정렬 오름차순 후 인덱싱)
# ynLabel.classes_
# featuresData["LE_INV_PROMOTION"] = ynLabel.inverse_transform( featuresData.LE_PROMOTION )
# featuresData = featuresData.drop(columns=["LE_INV_PROMOTION"] )

# 2. 특성 선정 및 데이터 분리

In [42]:
### 특성 선정

In [63]:
corrDf = featuresData.corr(method="pearson", numeric_only=True)
stdCorr = 0.3
label = ["QTY"]
features = list( corrDf.loc[ (abs( corrDf.QTY) >= stdCorr) &
                             (abs( corrDf.QTY) != 1)    ].index )
print( f"문제지 {features}, 정답지: {label} " )

문제지 ['HCLUS', 'PRO_PERCENT', 'LE_PROMOTION', 'LE_HOLIDAY'], 정답지: ['QTY'] 


In [66]:
### 데이터 분리 (쉽게 접근하는 방법)

In [86]:
trainData, testData = train_test_split( featuresData, test_size=0.2, random_state=1 )
## 훈련, 테스트데이터 내부에서 문제지 정답지 분리
trainingDataFeatures = trainData.loc[ :, features]
trainingDataLabel = trainData.loc[ :, label]
testDataFeatures = testData.loc[ :, features]
testDataLabel = testData.loc[ :, label]

In [87]:
print(trainingDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataFeatures.shape)
print(testDataLabel.shape)

(84, 4)
(84, 1)
(21, 4)
(21, 1)


# 3. 모델 적용

In [93]:
dtModel = tree.DecisionTreeRegressor( random_state= 1 )
fittedDtModel = dtModel.fit( trainingDataFeatures, trainingDataLabel )

# 4. 예측

In [101]:
dtPredictValue = fittedDtModel.predict( testDataFeatures )
len( dtPredictValue )

21

# 5. 데이터 정리

In [108]:
testDataLabel["PREDICT_DT"] = dtPredictValue
testDataLabel.head(2)

,QTY,PREDICT_DT
65,969,977.555556
35,2069,1448.400000


# 정확도 검증

In [110]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [112]:
maeDt = mean_absolute_error( testDataLabel.QTY, testDataLabel.PREDICT_DT )

In [115]:
rmseDt = np.sqrt( mean_squared_error( testDataLabel.QTY, testDataLabel.PREDICT_DT ) )

In [122]:
accuracyMatrix = pd.DataFrame( [ [maeDt, rmseDt]], columns = ["MAE_DT", "RMSE_DT"] )
accuracyMatrix

,MAE_DT,RMSE_DT
0,491.388889,571.356909
